In [6]:
from open3d import io
from open3d import geometry
from open3d import utility
from open3d import visualization
import meshcut
import scipy.interpolate as si
import splipy as sp
import numpy as np
import os
import math
import networkx as nx
import glob
import json
import ast
import copy
from geomdl import BSpline
from geomdl import multi
from geomdl import knotvector
from geomdl import operations
from geomdl import fitting
from geomdl.visualization import VisMPL
from ipywidgets import FloatProgress
from IPython.display import display

In [7]:
def create_pcd(points):
    pcd = geometry.PointCloud()
    pcd.points = utility.Vector3dVector(points)
    return pcd

def extract_landmarks(meshdir, landmarkscandir, landmarkdir, pomdir, percent_scan=0.25):
    mesh = io.read_triangle_mesh(meshdir)
    landmarkscans = glob.glob(f'{landmarkscandir}*.ply')
    mesh_vertices = np.asarray(mesh.vertices)
    pcd = create_pcd(mesh_vertices)
    tree = geometry.KDTreeFlann(pcd)
    landmarkdata = {}
    for filedir in landmarkscans:
        landmark_mesh = io.read_triangle_mesh(filedir)
        vertices = np.asarray(landmark_mesh.vertices)
        unordered_idx = []
        for vertex in vertices:
            [_, idx, d] = tree.search_knn_vector_3d(vertex, 1)
            unordered_idx.append([idx[0], d[0]])
        ordered_idx = np.asarray(sorted(unordered_idx, key=lambda x: x[1]))[:,0].astype(np.int32)
        percent_idx = ordered_idx[:int(len(ordered_idx)*percent_scan)+1]
        percent_points = mesh_vertices[percent_idx]
        name = filedir.split('_')[-1][:-4]
        landmark_mean = percent_points.mean(axis=0)
        [_, idx, _] = tree.search_knn_vector_3d(landmark_mean, 1)
        landmark_position = mesh_vertices[idx[0]]
        landmarkdata[name] = {
            'position': tuple(landmark_position)
        }
    with open(landmarkdir, 'w') as file:
        json.dump(landmarkdata, file, indent=4, sort_keys=True)

In [8]:
subjectdir = '/Users/nickf/Dropbox/Nicky T-Shirt Project/Subject Scans/Male Scans/Nicholas Ferrara/'
meshdir = f'{subjectdir}NF_SCAN_V3/NF_REGULAR_V1.4.1_M2_ADJ.ply'
landmarkscandir = f'{subjectdir}NF_LANDMARKS/'
landmarkdir = f'{subjectdir}NF_LANDMARKS.json'
pomdir = f'{subjectdir}NF_POMS_V2.json'
landmarkdata = extract_landmarks(meshdir, landmarkscandir, landmarkdir, pomdir, percent_scan=0.25)

In [65]:
def distance_tolerance(mesh, num_std):
    triangles = np.asarray(mesh.triangles)
    vertices = np.asarray(mesh.vertices)
    max_distances = []
    ordered_triangles = []
    for triangle in triangles:
        triangle_vertices = [vertices[vertex_idx] for vertex_idx in triangle]
        centroid = np.asarray(triangle_vertices).mean(axis=0)
        centroid_distances = []
        for vertex in triangle_vertices:
            diff_vector = centroid - vertex
            diff_length = np.sqrt(np.dot(diff_vector, diff_vector))
            centroid_distances.append(diff_length)
        max_centroid_distance = np.asarray(centroid_distances).max()
        max_distances.append(max_centroid_distance)
        ordered_triangles.append([centroid, triangle])
    tolerance = np.mean(max_distances) + num_std*np.std(max_distances)
    return ordered_triangles, tolerance

def read_scan(meshdir, landmarksdir, pomsdir):
    mesh = io.read_triangle_mesh(meshdir)
    mesh.compute_vertex_normals()
    
    with open(landmarksdir, 'r') as readlandmarks:
        landmarks = json.load(readlandmarks)
        with open(pomsdir, 'r') as readpoms:
            poms = json.load(readpoms)
            pom_landmarks = {}
            for key in poms:
                landmark_points = []
                for temp_key in poms[key]['landmarks']:
                    landmark_points.append(landmarks[temp_key]['position'])
                fitlandmark_points = []
                for temp_key in poms[key]['fitlandmarks']:
                    fitlandmark_points.append(landmarks[temp_key]['position'])
                landmark_constraints = []
                for temp_key in poms[key]['landmarks']:
                    landmark_constraints.append(landmarks[temp_key]['constraints'])
                landmark_dict = {
                    'fitlandmark_points': np.asarray(fitlandmark_points),
                    'landmark_points': np.asarray(landmark_points),
                    'periodic': poms[key]['periodic'],
                    'constraints': landmark_constraints,
                    'pom_alignment': [],
                    'section_normals': [],
                    'pom_points': None,
                    'pom_normals': None
                }
                pom_landmarks[key] = landmark_dict
    return mesh, pom_landmarks

def transform_wireframe(pom_landmarks_temp, key, delta):
    pom_landmarks = copy.deepcopy(pom_landmarks_temp)
    landmark_points = pom_landmarks[key]['landmark_points']
    landmark_constraints = pom_landmarks[key]['constraints']
    pom_points = pom_landmarks[key]['pom_points']
    pom_normals = pom_landmarks[key]['pom_normals']
    for landmark_idx, landmark_pt in enumerate(landmark_points):
        current_constraint = False
        next_constraint = False
        next_landmark_idx = landmark_idx + 1
        if pom_landmarks[key]['periodic']:
            if next_landmark_idx == len(landmark_points):
                next_landmark_idx = 0
                next_landmark_pom_idx = len(pom_points) - 1
            else:
                next_landmark_pom_idx = int(np.argwhere((pom_points == landmark_points[next_landmark_idx]).all(axis=1))[0])
        elif not next_landmark_idx == len(landmark_points):
            next_landmark_pom_idx = int(np.argwhere((pom_points == landmark_points[next_landmark_idx]).all(axis=1))[0])
        else:
            next_landmark_idx = 0
            next_landmark_pom_idx = int(np.argwhere((pom_points == landmark_points[0]).all(axis=1))[0])
        landmark_pom_idx = int(np.argwhere((pom_points == landmark_pt).all(axis=1))[0])
        
        landmark_normal = pom_normals[landmark_pom_idx]
        if landmark_constraints[next_landmark_idx][0] == 1:
            next_constraint = True
            c = delta/(next_landmark_pom_idx - landmark_pom_idx)
            current_transform = delta*landmark_normal
        elif landmark_constraints[landmark_idx][0] == 1:
            current_constraint = True
            c = delta/(next_landmark_pom_idx - landmark_pom_idx)
            current_transform = 0
        else:
            current_transform = delta*landmark_normal
        landmark_transformed = landmark_pt + current_transform
        pom_points[landmark_pom_idx] = landmark_transformed
        landmark_points[landmark_idx] = landmark_transformed
        current_pt_idx = landmark_pom_idx
        if pom_landmarks[key]['periodic']:
            while True:
                next_pt_idx = current_pt_idx + 1
                if landmark_idx == len(landmark_points) - 1 and next_pt_idx == len(pom_points):
                    break
                next_pt = pom_points[next_pt_idx]
                if (landmark_points == next_pt).all(axis=1).any():
                    break
                next_pt_normal = pom_normals[next_pt_idx]
                if current_constraint:
                    next_pt_transform = c*(next_pt_idx - landmark_pom_idx)
                elif next_constraint:
                    next_pt_transform = c*(next_landmark_pom_idx - next_pt_idx)
                else:
                    next_pt_transform = np.dot(current_transform, next_pt_normal) / np.dot(next_pt_normal, next_pt_normal)
                pom_points[next_pt_idx] = next_pt + next_pt_normal*next_pt_transform
                current_pt_idx += 1
                current_pt_normal = pom_normals[current_pt_idx]
                current_transform = delta*current_pt_normal
        elif landmark_idx < len(landmark_points) - 1:
            while True:
                next_pt_idx = current_pt_idx + 1
                next_pt = pom_points[next_pt_idx]
                if (landmark_points == next_pt).all(axis=1).any():
                    break
                next_pt_normal = pom_normals[next_pt_idx]
                if current_constraint:
                    next_pt_transform = c*(next_pt_idx - landmark_pom_idx)
                elif next_constraint:
                    next_pt_transform = c*(next_landmark_pom_idx - next_pt_idx)
                else:
                    next_pt_transform = np.dot(current_transform, next_pt_normal) / np.dot(next_pt_normal, next_pt_normal)
                pom_points[next_pt_idx] = next_pt + next_pt_normal*next_pt_transform
                current_pt_idx += 1
                current_pt_normal = pom_normals[current_pt_idx]
                current_transform = delta*current_pt_normal
    if pom_landmarks[key]['periodic']:
        landmark_idx = -1
        landmark_pt = landmark_points[landmark_idx]
        landmark_pom_idx = int(np.argwhere((pom_points == landmark_pt).all(axis=1))[0])
        landmark_normal = pom_normals[landmark_pom_idx]
        current_transform = delta*landmark_normal
        landmark_transformed = landmark_pt + current_transform
        pom_points[landmark_pom_idx] = landmark_transformed
    transformed_pcd = geometry.PointCloud()
    transformed_pcd.points = utility.Vector3dVector(pom_points)
    transformed_pcd.normals = utility.Vector3dVector(pom_normals)
    return transformed_pcd

def create_wireframe(mesh, pom_landmarks, num_std=10, num_neighbors=5):
    base_poms = []
    transformed_poms = []
    triangles = np.asarray(mesh.triangles)
    vertices = np.asarray(mesh.vertices)
    mesh_vertex_normals = np.asarray(mesh.vertex_normals)
    ordered_triangles, centroid_tolerance = distance_tolerance(mesh, num_std)
    f = FloatProgress(min=1, max=len(pom_landmarks.keys()))
    display(f)
    for key in pom_landmarks:
        landmark_points = pom_landmarks[key]['landmark_points']
        fitlandmark_points = pom_landmarks[key]['fitlandmark_points']
        if len(fitlandmark_points) > 0:
            fit_points = np.vstack((landmark_points, fitlandmark_points))
            section_mean = fit_points.mean(axis=0)
        else:
            section_mean = landmark_points.mean(axis=0)
        initial_landmark_idx = 0
        if pom_landmarks[key]['periodic']:
            initial_landmark_idx = -1
        pom_points = None
        for landmark_idx in range(initial_landmark_idx, len(landmark_points)-1):
            current_landmark = landmark_points[landmark_idx]
            next_landmark = landmark_points[landmark_idx+1]
            diff_vector1 = current_landmark - section_mean
            diff_vector2 = next_landmark - section_mean
            section_normal = np.cross(diff_vector1, diff_vector2)
            normal_length = np.sqrt(np.dot(section_normal, section_normal))
            pom_landmarks[key]['section_normals'].append(section_normal)
            # Project Triangle Centroid onto section plane to find the triangles within the
            # tolerance of maximum distance from the plane
            d = np.dot(section_normal, section_mean)
            valid_triangles = []
            for centroid, triangle in ordered_triangles:
                if abs(np.dot(section_normal, centroid) - d) / normal_length <= centroid_tolerance:
                    valid_triangles.append(triangle)
            valid_triangles = np.asarray(valid_triangles)
            # Slicing the mesh
            mesh_slices = meshcut.cross_section(vertices, valid_triangles,
                                                plane_orig=tuple(section_mean), plane_normal=tuple(section_normal))
            # Determine the slice by the presents of the landmarks
            current_slice = None
            current_landmark_idx = None
            next_landmark_idx = None
            for slice_section in mesh_slices:
                current_landmark_equality = (slice_section == current_landmark).all(axis=1)
                next_landmark_equality = (slice_section == next_landmark).all(axis=1)
                if current_landmark_equality.any() and next_landmark_equality.any():
                    current_slice = slice_section
                    current_landmark_idx = int(np.argwhere(current_landmark_equality))
                    next_landmark_idx = int(np.argwhere(next_landmark_equality)) 
            current_slice = np.vstack((current_slice[current_landmark_idx:], current_slice[:current_landmark_idx]))
            next_landmark_idx -= current_landmark_idx
            current_landmark_idx = 0
            if next_landmark_idx < 0:
                next_landmark_idx += len(current_slice)
            if next_landmark_idx < len(current_slice) - next_landmark_idx:
                current_slice = current_slice[:next_landmark_idx+1]
            else:
                current_slice = np.vstack((current_slice[next_landmark_idx:], current_slice[current_landmark_idx]))
                current_slice = np.flip(current_slice, axis=0)
            if type(pom_points) == type(None):
                pom_points = current_slice
            else:
                assert (pom_points[-1] == current_slice[0]).all(), f'{pom_points, current_slice}\nPrevious Section not connected by landmark'
                pom_points = np.vstack((pom_points[:-1], current_slice))
        # Storing vertex normals of the vertices within the valid triangles for normal estimation        
        valid_vertices = []
        for triangle in valid_triangles:
            for vertex_idx in triangle:
                valid_vertices.append(vertex_idx)
        vertices_idx = np.unique(np.asarray(valid_vertices))
        valid_vertices = np.asarray([vertices[vertex_idx] for vertex_idx in vertices_idx])
        valid_vertex_normals = np.asarray([mesh_vertex_normals[vertex_idx] for vertex_idx in vertices_idx])
        # Estimate pom points normals
        tree = geometry.KDTreeFlann(create_pcd(valid_vertices))
        pom_normals = []
        for point in pom_points:
            [_, neighbors_idx, _] = tree.search_knn_vector_3d(point, num_neighbors)
            normal = valid_vertex_normals[list(neighbors_idx)].mean(axis=0)
            # Projection of normal onto section normal
            normal = normal - section_normal*(np.dot(normal, section_normal)/np.dot(section_normal, section_normal)) 
            pom_normals.append(normal)
        pom_normals = np.asarray(pom_normals)
        pom_landmarks[key]['pom_points'] = pom_points
        pom_landmarks[key]['pom_normals'] = pom_normals
        # Base Pom Point Cloud
        pcd_base = geometry.PointCloud()
        pcd_base.points = utility.Vector3dVector(pom_points)
        pcd_base.normals = utility.Vector3dVector(pom_normals)
        base_poms.append(pcd_base)
        
        f.value += 1
    return base_poms

In [17]:
datafolderdir = '/Users/nickf/Dropbox/Nicky T-Shirt Project/Subject Scans/Male Scans/Nicholas Ferrara/'
meshfolderdir = '/Users/nickf/Dropbox/Nicky T-Shirt Project/Subject Scans/Male Scans/Nicholas Ferrara/NF_SCAN_V3/'
meshdir = f'{meshfolderdir}NF_REGULAR_V1.4.1_M2_ADJ.ply'
landmarksdir = f'{datafolderdir}NF_LANDMARKS.json'
pomsdir = f'{datafolderdir}NF_POMS_V2.json'
mesh, pom_landmarks = read_scan(meshdir, landmarksdir, pomsdir)

In [18]:
wireframe = create_wireframe(mesh, pom_landmarks, num_std=3, num_neighbors=5)

FloatProgress(value=1.0, max=23.0, min=1.0)

In [12]:
visualization.draw_geometries(wireframe)

In [66]:
transformed_wireframe = []
delta = 30
for key in pom_landmarks:
    transformed_wireframe.append(transform_wireframe(pom_landmarks, key, delta))

In [68]:
visualization.draw_geometries([mesh]+wireframe+transformed_wireframe)